# Apartment search using ORS and Ohsome API 


In this notebook, we will perform a search for a new appartment in Hamburg based on the surrounding amenities. 

Execute each cell one by one first to see how it works. Afterwards, you can go through the notebook again and change some of paramters in the cell where it says "**Exercise**".

In [1]:
# These are just some imports, which means loading additional functionalities in Python. Just ignore this. 
from ipyleaflet import Map, Marker, GeoData, basemaps,ScaleControl, LayersControl, GeoJSON
from ipywidgets import Layout
import json
import os
import pandas as pd
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

/Users/chludwig/Documents/UniHD/teaching/FOSSGIS/ws2122/assignments/fossgis_bonus/.venv/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


## Create a list of potential apartments

We store the locations of three potential appartments which we may have found online in a dictionary with geographic coordinates.

**Exercise:** Change the coordinates of the locations or add additinal ones. 

In [2]:
apartments = {'0': {'location': [9.981727, 53.552018]}, # longitude, latitude
              '1': {'location': [9.921727, 53.552018]},
              '2': {'location': [9.981727, 53.572018]}}

Let's convert it into a table.

In [3]:
apartments_df = gpd.GeoDataFrame().from_dict(apartments, orient="index")
apartments_df["geometry"] = apartments_df.location.map(lambda x: Point(x[0], x[1]))

In [4]:
apartments_df

,location,geometry
0,"[9.981727, 53.552018]",POINT (9.98173 53.55202)
1,"[9.921727, 53.552018]",POINT (9.92173 53.55202)
2,"[9.981727, 53.572018]",POINT (9.98173 53.57202)


### Map of apartments

**Exercise:** Change some of the parameters which define the map style, e.g. the color or radius of the points. You may also change the basemap. Refer to the documentation for [available basemaps](https://ipyleaflet.readthedocs.io/en/latest/api_reference/basemaps.html).

In [5]:
# Create map with basemap
map_center = list(apartments_df.dissolve().centroid[0].coords)[0]
m = Map(center=map_center[::-1], zoom=13, scroll_wheel_zoom=True, basemap=basemaps.CartoDB.Positron)

# Add appartments as points
apartment_layer = GeoData(geo_dataframe = apartments_df,
    style={'color': 'black', 
           'radius':8, 
           'fillColor': '#3366cc', 
           'opacity':0.5, 
           'weight':1.9, 
           'dashArray':'2', 
           'fillOpacity':0.6},
    hover_style={'fillColor': 'red' , 
                 'fillOpacity': 0.2},
    point_style={'radius': 5, 
                 'color': 'red', 
                 'fillOpacity': 0.8, 
                 'fillColor': 'blue', 
                 'weight': 3},
    name = 'Apartments')
m.add_layer(apartment_layer)

# Add a scale to map 
m.add_control(ScaleControl(position='bottomleft'))
# Add layer control 
control = LayersControl(position='topright')
m.add_control(control)

display(m)

Map(center=[53.558684666666664, 9.961727], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in…

## Count number of restaurants nearby each appartment

Let's assume it is very important to you that you have many restaurants in walking distance of your apartment. So we will count how many restaurants are within a 500 meter walking distance of each appartment. There are two steps: 

1. Calculate the area within 500 meter walking distance around each appartment using openrouteservice isochrones
2. Downloading and coubnting the restaurants from OpenStreetMap using the ohsome API. 

### 1. Create isochrones around each apartment

First, we will request the isochrones around each apartment and visualize them in the map. For details about the parameters see the [ORS documentation](https://openrouteservice.org/dev/#/api-docs/isochrones). Play around with the settings to find out what each variable is for.

In order to use the OpenRouteSerive API, you need to [create an account and generate a free API key](https://openrouteservice.org/dev/#/login). You can use your github account for it.

In [7]:
from openrouteservice import client

In [6]:
ors_api_key = '5b3ce3597851110001cf6248eef794d1244544f7826f417356aee9e4' #Replace this with your own personal ORS API key

In [8]:
ors_client = client.Client(key=ors_api_key) 

**Execise:** Switch the routing profile to cycling by replacing `foot-walking` with `cycling-regular` and run the cell below again. 

In [21]:
ors_settings = {'profile': 'foot-walking', # ORS routing profile
              'intervals': [500], # Interval of isochrones in min if range_type=time
              'segments': 500, 
              'range_type':  'time', #Type of isochones: time or distance
              'attributes': ['area'], # Get area of each isochrone
              'locations' : list(apartments_df.location)
             }

In [10]:
isochrones = ors_client.isochrones(**ors_settings) 

#### Map with appartments and isochfrom_dict

In [11]:
# Create map with basemap (available basemaps: https://ipyleaflet.readthedocs.io/en/latest/api_reference/basemaps.html)
m = Map(center=map_center[::-1], zoom=13, scroll_wheel_zoom=True, basemap=basemaps.CartoDB.Positron)

# Add isochrones 
isochones_layer = GeoJSON(
    data=isochrones,
    style={
        'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
    })
m.add_layer(isochones_layer)

# Add appartments as points
apartment_layer = GeoData(geo_dataframe = apartments_df,
    style={'color': 'black', 
           'radius':8, 
           'fillColor': '#3366cc', 
           'opacity':0.5, 
           'weight':1.9, 
           'dashArray':'2', 
           'fillOpacity':0.6},
    hover_style={'fillColor': 'red' , 
                 'fillOpacity': 0.2},
    point_style={'radius': 5, 
                 'color': 'red', 
                 'fillOpacity': 0.8, 
                 'fillColor': 'blue', 
                 'weight': 3},
    name = 'Apartments')
m.add_layer(apartment_layer)

# Add a scale to map 
m.add_control(ScaleControl(position='bottomleft'))

control = LayersControl(position='topright')
m.add_control(control)

display(m)

Map(center=[53.558684666666664, 9.961727], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in…

## 2. Number of restaurants nearby apartments using Ohsome API
Now that we created the isochrones, we want to know how many restaurants there are in the neighbourhood. For this purpose, we will use the Ohsome API. 

In [12]:
import ohsome
ohsome_client = ohsome.OhsomeClient()

### 2.1. Download restaurants as features to visualized them on the map

**Exercise:** Set a different tag in the the `osm_filter`. 

In [13]:
osm_filer = "amenity=restaurant"

In [14]:
restaurant_features = ohsome_client.elements.centroid.post(bpolys=json.dumps(isochrones),
                                                           properties="tags",
                                                             filter=osm_filer)
restaurants_df = restaurant_features.as_dataframe()

In [15]:
restaurants_df.head()

,,geometry,addr:city,addr:country,addr:housenumber,addr:postcode,addr:street,amenity,buildingpart,floor,name,...,description,internet_access,layer,vegetarian,fixme,opening_hours:covid19,contact:email,bar,image,entrance
@osmId,@snapshotTimestamp,,,,,,,,,,,,,,,,,,,,,
node/1077489885,2022-02-07 15:00:00,POINT (9.92646 53.55541),Hamburg,DE,34a,22765,Friedensallee,restaurant,NaN,NaN,El Iberico,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
node/1078576284,2022-02-07 15:00:00,POINT (9.92701 53.55040),Hamburg,DE,42,22765,Eulenstraße,restaurant,NaN,NaN,Brasserie La Provence,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
node/1162424649,2022-02-07 15:00:00,POINT (9.92304 53.54823),NaN,NaN,NaN,NaN,NaN,restaurant,NaN,NaN,HACO Greenhouse,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
node/1342652460,2022-02-07 15:00:00,POINT (9.92890 53.55399),NaN,NaN,NaN,NaN,NaN,restaurant,NaN,NaN,Piazza Italiana,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
node/1357756659,2022-02-07 15:00:00,POINT (9.92881 53.55433),Hamburg,DE,170,22765,Bahrenfelder Straße,restaurant,NaN,NaN,Wild Rice,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2.2. Intersect isochrones with restaurants and count them

We convert the isochrones into a dataframe called `isochrones_df` and intersect it with the dataframe `restaurants_df` containing the restaurants.

In [16]:
isochones_df = gpd.GeoDataFrame.from_features(isochrones['features'], crs="epsg:4326")
isochrone_restaurants = gpd.sjoin(restaurants_df, isochones_df, how="left").groupby("group_index").count()
apartments_df["restaurants"] = list(isochrone_restaurants["amenity"])

In [17]:
apartments_df

,location,geometry,restaurants
0,"[9.981727, 53.552018]",POINT (9.98173 53.55202),73
1,"[9.921727, 53.552018]",POINT (9.92173 53.55202),23
2,"[9.981727, 53.572018]",POINT (9.98173 53.57202),39


### Alternative:  Count number of restaurants within isochrones using ohsome API

Alternatively, we could have calculated the number of restauratans within in each isochrone also using the ohsome API directly without downloading the restaurant features. 

In [18]:
response = ohsome_client.elements.count.groupByBoundary.post(bpolys=json.dumps(isochrones),
                                                             filter="amenity=restaurant")
restaurants = response.as_dataframe()["value"]
apartments_df["restaurants_ohsome"] = list(restaurants)

In [19]:
apartments_df

,location,geometry,restaurants,restaurants_ohsome
0,"[9.981727, 53.552018]",POINT (9.98173 53.55202),73,73.0
1,"[9.921727, 53.552018]",POINT (9.92173 53.55202),23,23.0
2,"[9.981727, 53.572018]",POINT (9.98173 53.57202),39,39.0


### Map with apartments and restaurants

In [20]:
# Create map with basemap (available basemaps: https://ipyleaflet.readthedocs.io/en/latest/api_reference/basemaps.html)
m = Map(center=map_center[::-1], zoom=13, scroll_wheel_zoom=True, layout=Layout(width='100%', height='500px'),
        basemap=basemaps.CartoDB.Positron)

# Add isochrones 
isochones_layer = GeoJSON(
    data=isochrones,
    style={
        'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
    })
m.add_layer(isochones_layer)

# Add appartments as points
apartment_layer = GeoData(geo_dataframe = apartments_df,
    style={'color': 'black', 
           'radius':8, 
           'fillColor': '#3366cc', 
           'opacity':0.5, 
           'weight':1.9, 
           'dashArray':'2', 
           'fillOpacity':0.6},
    hover_style={'fillColor': 'red' , 
                 'fillOpacity': 0.2},
    point_style={'radius': 5, 
                 'color': 'red', 
                 'fillOpacity': 0.8, 
                 'fillColor': 'blue', 
                 'weight': 3},
    name = 'Apartments')
m.add_layer(apartment_layer)

# Add restaurants as points
restaurants_layer = GeoData(geo_dataframe = restaurants_df,
    style={'color': 'black', 
           'radius':4, 
           'fillColor': 'red', 
           'opacity':0.5, 
           'weight':1.9, 
           'dashArray':'2', 
           'fillOpacity':0.6},
    hover_style={'fillColor': 'red' , 
                 'fillOpacity': 0.2},
    point_style={'radius': 5, 
                 'color': 'red', 
                 'fillOpacity': 0.8, 
                 'fillColor': 'blue', 
                 'weight': 3},
    name = 'Restaurants')

m.add_layer(restaurants_layer)

# Add a scale to map 
m.add_control(ScaleControl(position='bottomleft'))

control = LayersControl(position='topright')
m.add_control(control)

display(m)

Map(center=[53.558684666666664, 9.961727], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in…